In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/My Drive/fake-news/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(18285, 5)

In [9]:
##getting the title column seperately
newstext=df['title']

In [10]:
newstext.head(7)

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
5    Jackie Mason: Hollywood Would Love Trump if He...
7    Benoît Hamon Wins French Socialist Party’s Pre...
Name: title, dtype: object

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import re
from nltk.corpus import stopwords

In [13]:
newstextcopy=newstext.copy()

In [14]:
newstextcopy.reset_index(drop=True,inplace=True)

In [15]:
newstextcopy[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [16]:
from nltk.stem.porter import PorterStemmer

In [17]:
ps=PorterStemmer()
corpus=[]
vocabulary_size=10000

In [18]:
for i in range(0,len(newstextcopy)):
  stemnews=re.sub('[^a-zA-Z]',' ',newstextcopy[i])
  stemnews=stemnews.lower()
  stemnews=stemnews.split()
  stemnews= [ps.stem(word) for word in stemnews if not word in stopwords.words('english')]
  stemnews = ' '.join(stemnews)
  corpus.append(stemnews)

In [19]:
newstextcopy[5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

In [ ]:
newstextcopy.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
from nltk.tokenize import word_tokenize
sample=word_tokenize(corpus[2])
len(sample)

4

In [23]:
import tensorflow as tf

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [25]:
vocabulary_size=10000

In [26]:
onehot_repr=[one_hot(words,vocabulary_size)for words in corpus] 
onehot_repr

[[2887, 6039, 7618, 1014, 5360, 9771, 5302, 3783, 1259, 4701],
 [7116, 6020, 4066, 8564, 8655, 4324, 9608],
 [1222, 4018, 5906, 1746],
 [6225, 3436, 9183, 4660, 166, 8570],
 [5252, 8655, 1671, 359, 1088, 8641, 8655, 7134, 6887, 6433],
 [4433,
  3285,
  934,
  3787,
  1358,
  7036,
  3725,
  906,
  7264,
  4827,
  9821,
  6420,
  5301,
  3567,
  9608],
 [6896, 2700, 3622, 7595, 6308, 9159, 9653, 8117, 4322, 2634, 5832],
 [3549, 6091, 7919, 7165, 2526, 9890, 7036, 7606, 4322, 2634, 5832],
 [5849, 7661, 5903, 2436, 4030, 5214, 3909, 5959, 7036, 8145],
 [4580, 2570, 2566, 9630, 2775, 5207, 970, 3946],
 [73, 4892, 4707, 7189, 1469, 6334, 6288, 1879, 5364, 2533, 9929],
 [4660, 19, 5360, 5214, 7036, 2526],
 [1205, 1918, 583, 1202, 7857, 3833, 8432, 570, 7958],
 [9194, 3036, 4683, 252, 9783, 1899, 5827, 4322, 2634, 5832],
 [3913, 9111, 9942, 831, 9858, 4322, 2634, 5832],
 [1457, 7978, 8463, 263, 1559, 9203, 1991, 4761, 9415, 947],
 [4599, 3635, 6020],
 [3640, 902, 875, 3922, 7036, 4584, 7604, 

In [27]:
maxtext=0
for items in onehot_repr:
  maxtext=max(maxtext,len(items))
print(maxtext)

47


In [28]:
sentence_length=50
embedded_text=pad_sequences(onehot_repr,padding='pre',maxlen=sentence_length)
print(embedded_text)

[[   0    0    0 ... 3783 1259 4701]
 [   0    0    0 ... 8655 4324 9608]
 [   0    0    0 ... 4018 5906 1746]
 ...
 [   0    0    0 ... 4322 2634 5832]
 [   0    0    0 ... 4092 4927 5889]
 [   0    0    0 ... 1058  353 8763]]


In [29]:
embedded_text[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1222, 4018, 5906, 1746], dtype=int32)

In [30]:
y=df['label']

In [31]:
len(newstextcopy)

18285

In [32]:
len(y)

18285

In [33]:
import numpy as np
X_final=np.array(embedded_text)
y_final=np.array(y)

In [34]:
X_final.shape,y_final.shape

((18285, 50), (18285,))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)

In [ ]:
embedding_vector_features=30
model2=Sequential()
model2.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sentence_length))
model2.add(LSTM(100))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 30)            300000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               52400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 352,501
Trainable params: 352,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
192/192 [==============================] - 14s 72ms/step - loss: 0.3479 - accuracy: 0.8340 - val_loss: 0.2012 - val_accuracy: 0.9133
Epoch 2/5
192/192 [==============================] - 13s 70ms/step - loss: 0.1319 - accuracy: 0.9483 - val_loss: 0.1944 - val_accuracy: 0.9236
Epoch 3/5
192/192 [==============================] - 14s 71ms/step - loss: 0.0782 - accuracy: 0.9734 - val_loss: 0.2049 - val_accuracy: 0.9160
Epoch 4/5
192/192 [==============================] - 14s 70ms/step - loss: 0.0487 - accuracy: 0.9846 - val_loss: 0.2299 - val_accuracy: 0.9181
Epoch 5/5
192/192 [==============================] - 14s 71ms/step - loss: 0.0263 - accuracy: 0.9927 - val_loss: 0.3660 - val_accuracy: 0.9205


In [ ]:
y_pred2=model2.predict_classes(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      3389
           1       0.88      0.95      0.91      2646

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



In [40]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [41]:
def create_model(layers,embedding_vector_features):
  model4=Sequential()
  model4.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sentence_length))
  model4.add(LSTM(layers))
  model4.add(Dense(1,activation='sigmoid'))
  model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model4
model4=KerasClassifier(build_fn=create_model,verbose=0)

In [42]:
model4

In [45]:
layers=[100,150]
embedding_vector_features=[30,40]
param_grid=dict(layers=layers,embedding_vector_features=embedding_vector_features,batch_size=[32,64],epochs=[5,10])
grid= GridSearchCV(estimator=model4,param_grid=param_grid)

In [46]:
grid_result=grid.fit(X_train, y_train)

In [47]:
[grid_result.best_score_,grid_result.best_params_]

[0.9159183740615845,
 {'batch_size': 32,
  'embedding_vector_features': 40,
  'epochs': 5,
  'layers': 150}]

In [48]:
embedding_vector_features=40
bestmodel=Sequential()
bestmodel.add(Embedding(vocabulary_size,embedding_vector_features,input_length=sentence_length))
bestmodel.add(LSTM(150))
bestmodel.add(Dense(1,activation='sigmoid'))
bestmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(bestmodel.summary())

Model: "sequential_99"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_99 (Embedding)     (None, 50, 40)            400000    
_________________________________________________________________
lstm_99 (LSTM)               (None, 150)               114600    
_________________________________________________________________
dense_99 (Dense)             (None, 1)                 151       
Total params: 514,751
Trainable params: 514,751
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
bestmodel.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=32)

Epoch 1/5
383/383 [==============================] - 5s 13ms/step - loss: 0.2667 - accuracy: 0.8784 - val_loss: 0.1797 - val_accuracy: 0.9243
Epoch 2/5
383/383 [==============================] - 4s 11ms/step - loss: 0.1165 - accuracy: 0.9567 - val_loss: 0.1901 - val_accuracy: 0.9264
Epoch 3/5
383/383 [==============================] - 4s 11ms/step - loss: 0.0682 - accuracy: 0.9768 - val_loss: 0.2179 - val_accuracy: 0.9249
Epoch 4/5
383/383 [==============================] - 5s 12ms/step - loss: 0.0422 - accuracy: 0.9865 - val_loss: 0.2665 - val_accuracy: 0.9238
Epoch 5/5
383/383 [==============================] - 5s 12ms/step - loss: 0.0272 - accuracy: 0.9913 - val_loss: 0.2932 - val_accuracy: 0.9173


In [62]:
y_pred=bestmodel.predict_classes(X_test)

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3389
           1       0.91      0.90      0.91      2646

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3158,  231],
       [ 268, 2378]])

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9173156586578294